<a href="https://colab.research.google.com/github/tobiolad/Working-with-PythonSQL/blob/main/LinkedIn_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google googlesearch-python beautifulsoup4

  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=15794551d40d59f81a95efdf44c6de03c228fa989dd7ae733216f0fc8ed1f46c
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


In [ ]:
pip install selenium beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [17]:
import requests
import re
import csv
import os
from time import sleep
import pandas as pd
import numpy as np
from googlesearch import search
from bs4 import BeautifulSoup

# Define the sanitize_filename function
def sanitize_filename(filename):
    # Replace invalid characters with underscores
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

class LinkedIn:
    def __init__(self):
        self.fieldnames = ["Profile Link", "Name", "Designation", "Location"]

    def saveData(self, dataset, output_file_name):
        with open(output_file_name, mode='a+', encoding='utf-8-sig', newline='') as csvFile:
            writer = csv.DictWriter(
                csvFile, fieldnames=self.fieldnames, delimiter=',', quotechar='"')
            if os.stat(output_file_name).st_size == 0:
                writer.writeheader()
            writer.writerow({
                "Profile Link": dataset[0],
                "Name": dataset[1],
                "Designation": dataset[2],
                "Location": dataset[3]
            })

    @classmethod
    def getCompanyID(cls, company_link):
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'max-age=0',
            'Dnt': '1',
            'Sec-Ch-Ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"Windows"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        }
        try:
            resp = requests.get(company_link, headers=headers).text
        except:
            print("Failed to open {}".format(company_link))
            return None
        try:
            companyID = re.findall(
                r'"objectUrn":"urn:li:organization:([\d]+)"', resp)[0]
        except:
            print("Company ID not found")
            return None
        return companyID

    def paginateResults(self, companyID, cookies, pagination_delay, output_file_name):
        csrf_tokens = re.findall(r'JSESSIONID="(.+?)"', cookies)
        csrf_token = csrf_tokens[0] if csrf_tokens else ''
        headers = {
            'Accept': 'application/vnd.linkedin.normalized+json+2.1',
            'Cookie': cookies,
            'Csrf-Token': csrf_token,
            'Dnt': '1',
            'Referer': 'https://www.linkedin.com/search/results/people/?currentCompany=%5B%22' + companyID + '%22%5D&origin=COMPANY_PAGE_CANNED_SEARCH&page=2&sid=7Gd',
            'Sec-Ch-Ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"Windows"',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
            'X-Li-Lang': 'en_US',
            'X-Li-Page-Instance': 'urn:li:page:d_flagship3_search_srp_people_load_more;Ux/gXNk8TtujmdQaaFmrPA==',
            'X-Li-Track': '{"clientVersion":"1.13.9792","mpVersion":"1.13.9792","osName":"web","timezoneOffset":6,"timezone":"Asia/Dhaka","deviceFormFactor":"DESKTOP","mpName":"voyager-web","displayDensity":1.3125,"displayWidth":1920.1875,"displayHeight":1080.1875}',
            'X-Restli-Protocol-Version': '2.0.0',
        }
        for page_no in range(0, 10, 10):
            print("Checking facet: {}/990".format(page_no))
            link = "https://www.linkedin.com/voyager/api/graphql?variables=(start:" + str(page_no) + ",origin:COMPANY_PAGE_CANNED_SEARCH,query:(flagshipSearchIntent:SEARCH_SRP,queryParameters:List((key:currentCompany,value:List(" + \
                companyID + \
                ")),(key:resultType,value:List(PEOPLE))),includeFiltersInResponse:false))&queryId=voyagerSearchDashClusters.e1f36c1a2618e5bb527c57bf0c7ebe9f"

            try:
                resp = s.get(link, headers=headers).json()
            except:
                print("Failed to open {}".format(link))
                continue
            results = resp.get('included')
            for person_data in results:
                if person_data.get('$type') == "com.linkedin.voyager.dash.search.EntityResultViewModel":
                    person_name = person_data.get('title').get('text')
                    profile_link = person_data.get('navigationUrl')

                    # Check if primarySubtitle is None before accessing its text attribute
                    primary_subtitle = person_data.get('primarySubtitle')
                    designation = primary_subtitle.get('text') if primary_subtitle else None

                    # Similarly, check if secondarySubtitle is None before accessing its text attribute
                    secondary_subtitle = person_data.get('secondarySubtitle')
                    person_location = secondary_subtitle.get('text') if secondary_subtitle else None

                    print("Profile Link: {}".format(profile_link))
                    print("Name: {}".format(person_name))
                    print("Designation: {}".format(designation))
                    print("Location: {}".format(person_location))
                    print()

                    dataset = [profile_link, person_name, designation, person_location]
                    self.saveData(dataset, output_file_name)
            print("Waiting for {} seconds".format(pagination_delay))
            sleep(pagination_delay)

def google_search(query, num_results=10, lang='en'):
    """
    Perform a Google search and return a list of URLs.
    """
    search_results = search(query, num_results=num_results, lang=lang)
    return search_results

def get_linkedin_url(company_name):
    """
    Search Google for the company's LinkedIn profile and scrape the URL of the first result.
    """
    query = f"{company_name} LinkedIn"
    search_results = google_search(query)

    for url in search_results:
        if "linkedin.com/company" in url:
            return url

    return None

def scrape_linkedin_url(company_name):
    """
    Scrape the URL of the company's LinkedIn profile.
    """
    linkedin_url = get_linkedin_url(company_name)
    if linkedin_url:
        response = requests.get(linkedin_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return linkedin_url
    else:
        return None

s = requests.Session()

# Example usage:
file_name = pd.read_csv('./linkedin_scraping.csv') # put file directory here
company_names = file_name['Name'].values[191:327]

for company_name in company_names:
    company_link = scrape_linkedin_url(company_name)
    if company_link:
        print(f"The LinkedIn URL for {company_name} is: {company_link}")
    else:
        print(f"No LinkedIn profile found for {company_name}.")
        continue

    output_file_name = sanitize_filename(f"{company_name}.csv")  # output CSV excel file name
    pagination_delay = 5  # delay in seconds before going to the next page
    cookies = 'li_sugr=d58f1b94-64ef-406a-a881-13b7207f39e0; bcookie="v=2&a3745c43-8f27-4c0b-8867-4e8192a2738e"; bscookie="v=1&20240106221724031e9c6e-1a34-4ce7-85b6-b0620aa6ca90AQHShBCFTf0Wtb7OaFI-A-cV1AAcYvaZ"; JSESSIONID="ajax:4470063861007695657"; timezone=America/Toronto; li_theme=light; li_theme_set=app; _guid=e666fbaf-73fb-42ed-a14c-1e27dac787cf; aam_uuid=33147409925437713830234114304413153587; _gcl_au=1.1.234055212.1704585726; liap=true; dfpfpt=bbe26affd46d4a94b72253d36c29c7a3; li_at=AQEDATZ9R1EEbu3jAAABjUjy9-YAAAGN3h4_sk0ADBtX8e4WxsjhwPPQsWASxe3vkvF6_4Lq0uK06Wx_6VHRCj9c3kGyxxSHcs9hN2FZer1k_deBSbkVhDUcQeRbOF10uaIyo7trutvRUWNL6RaMB7vZ; gpv_pn=www.linkedin.com%2Fcompany%2Fid-redacted%2Fadmin%2Fanalytics%2Ffollowers%2F; s_ips=830.1999969482422; s_tslv=1708397158247; s_tp=2970; lang=v=2&lang=en-us; fptctx2=taBcrIH61PuCVH7eNCyH0F58uBDuZFZOunQHZt3FuglNSBz0kNWLrWQTo0Un%252f%252bynhuoqWYITZl7djIDr35X8NvU7Ef%252bwPUfmSdl%252bZiH6zjqMrEvyDwrTrTrZu%252ffj82G5X0IVl%252bBbtsl5uc4iH58rQudbasEO8F3ETBjvEmU1md%252by1E%252bNhCq7LXsQfiIUWMpTh0YKfFIJ5X8Z5q0KM6JeNrBNMBpMwa7pLG%252fdAIOwLrESC35Pconqc2VarYoZpwfrqgjm%252fEgc7w%252fKsglHp8L8KGB2uY3DDqb3mtc%252fdFo7C0WNtoJLJGHl0BphZReYJa5OBFzThVqkMZVfMrQsTYsovPxJq60B25EGpW6XVmcM27g%253d; AMCVS_14215E3D5995C57C0A495C55%40AdobeOrg=1; AMCV_14215E3D5995C57C0A495C55%40AdobeOrg=-637568504%7CMCIDTS%7C19775%7CMCMID%7C33730514080517143640283887874633419512%7CMCAAMLH-1709230877%7C7%7CMCAAMB-1709230877%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1708633277s%7CNONE%7CvVersion%7C5.1.1%7CMCCIDH%7C2007872224; UserMatchHistory=AQKMHpYxjqzFugAAAY3SOAIMTKS-RHKaYk5f68mSeRluDHEeDs-UmxLpovJApjYxod7vphXqeDNeHxDl5AiXeH7FdVd9W12uJt8XdClCVa3nGUt22y-HNebEo3Az585N8XJSfG61eJoG3FabphQyzToxHHSHnghRN4fHInfMbWsv-PGDdub2vqNS4lZBem8jJ4jD6WiA8S_r0HZd3a29LcF5g-Yh24hETmUx8wXXXVamp-czl4s2blvgdIzeEBJuHl8v6fnyg6-caiPRDC7sK6SKq07-bMbJQbpQy10LF46SLTHsRICkpHJy7b1uw3AepPAl06o; AnalyticsSyncHistory=AQLLiC8X_379BgAAAY3SOAIMUu5MKztoITkH6U3Pv1SxqwKPUxF5RGMVO91Q-o4Q8xkNdnNlpPno51kfPC9-Uw; lms_ads=AQEpnPYWQEA4EwAAAY3SOAJc6DZB1IT4PNWH63tVr7yd7qxvzPU4rhajhVItUDaMAMF6WpPSV69wY1zl94zgALnI678DpDdR; lms_analytics=AQEpnPYWQEA4EwAAAY3SOAJc6DZB1IT4PNWH63tVr7yd7qxvzPU4rhajhVItUDaMAMF6WpPSV69wY1zl94zgALnI678DpDdR; lidc="b=VB21:s=V:r=V:a=V:p=V:g=5296:u=813:x=1:i=1708628902:t=1708632183:v=2:sig=AQECmq4zsEcAVmlZ939tDqMOIbSkqnTd"'  # place cookie here

    if __name__ == "__main__":
        companyID = LinkedIn.getCompanyID(company_link)
        if companyID is not None:
            linkedin = LinkedIn()
            linkedin.paginateResults(companyID, cookies, pagination_delay, output_file_name)
        else:
            continue


HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DMistral%252BVenture%252BPartners%252BInc.%252BLinkedIn%26num%3D12%26hl%3Den%26start%3D0&hl=en&q=EgQiW4KAGJ6Z364GIjAz9H2Mbl0kSQcSNG64WYe-UnkgSpqNkZ2dIGRsaTfxRpB-e0k6I3NGhEudD-fXc90yAXJaAUM

In [18]:
import os
import pandas as pd

# Get the list of CSV files in the directory
csv_files = [file for file in os.listdir('/content') if file.endswith('.csv') and file != 'linkedin_scraping.csv']

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file and append its DataFrame to the list
for file in csv_files:
    df = pd.read_csv(os.path.join('/content', file))
    df['Category'] = file[:-4]  # Add a category column based on the filename
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Display the merged DataFrame
print(merged_df)

# Optionally, you can save the merged DataFrame to a new CSV file
merged_df.to_csv('/content/merged_data.csv', index=False)


                                           Profile Link  \
0     https://www.linkedin.com/in/krishna-shah07?min...   
1     https://www.linkedin.com/in/annie-zou-50764b17...   
2     https://www.linkedin.com/in/daniel-dampf-362b4...   
3     https://www.linkedin.com/in/malency-wainwright...   
4     https://www.linkedin.com/in/shenna-marie-bongg...   
...                                                 ...   
1544  https://www.linkedin.com/in/simranassi95?miniP...   
1545  https://www.linkedin.com/in/wendy-lockwood-7b9...   
1546  https://www.linkedin.com/in/apetroucic?miniPro...   
1547  https://www.linkedin.com/in/tania-bickerstaffe...   
1548  https://www.linkedin.com/in/mike-mannix?miniPr...   

                                   Name  \
0                          Krishna Shah   
1                             Annie Zou   
2                          Daniel Dampf   
3     Malency Wainwright, MBA, CPA, CMA   
4                  Shenna Marie Bonggot   
...                              